In [57]:
import pandas as pd
import openai
from typing import List, Dict
import re
import json
from datetime import datetime
import copy
import numpy as np
from tqdm import tqdm
import time
import random

In [58]:
config_file_path = 'params.json'

with open(config_file_path, 'r') as config_file:
    config_data = json.load(config_file)
    config = config_data.get('config', {})

In [32]:
# def generate_prompt(vertical: str, sub_category: str, tags: list) -> str:
#     """Generate a prompt based on the vertical, sub category, and tags."""
    
#     # Format tags if the list is not empty
#     tag_instructions = f" Consider these concepts: {', '.join(tags)}." if tags else ""
    
#     question_starters = [
#         "What's your take on",
#         "How do you approach",
#         "What challenges arise with",
#         "Share your experience with",
#         "What innovations excite you in",
#         "How do you see the future of",
#         "What strategies work best for",
#         "What's often overlooked about",
#         "How has your perspective changed on",
#         "What advice would you give about"
#     ]
    
#     return f"""Create a concise, open-ended question about {vertical} - {sub_category}.{tag_instructions}
#     The question should:
#     1. Invite professional insights and personal experiences.
#     2. Avoid simple facts or yes/no answers.
#     3. Use natural, varied language.
#     4. Be 90-110 characters on average.
#     5. Be in clear, correct English.

#     Example: "{random.choice(question_starters)} {sub_category} in {vertical}?"

#     Vary your phrasing and structure to avoid repetitive or formulaic questions."""

In [54]:
def generate_prompt(vertical: str, sub_category: str, tags: list) -> str:
    """Generate a prompt based on the vertical, sub category, and tags."""
    tag_text = f" Consider these concepts: {', '.join(tags)}." if tags else ""
    
    return f"""Create a unique, engaging question about {vertical} - {sub_category}.{tag_text}

    Key points:
    - Aim for 90-110 characters on average
    - Be unpredictable in structure and wording
    - Avoid any formulaic or robotic-sounding phrases
    - Seek professional insights or personal experiences
    - Use natural, conversational language
    - Ensure it's open-ended and not easily Googleable
    - Mix up question types
    - Surprise the reader with unexpected angles or perspectives
    
    Example: "Ever had a {sub_category} idea backfire in {vertical}? What's your craziest story?"
    
    Remember: Each question should feel fresh and distinct from others in the same category."""

In [34]:
# def generate_prompt(vertical: str, sub_category: str, tags: list) -> str:
#     """Generate a prompt based on the vertical, sub category, and tags."""
#     tag_instructions = f", incorporating these concepts as relevant: {', '.join(tags)}." if tags else "."
    
#     return f"""Create a brief, open-ended question about {vertical} - {sub_category}{tag_instructions} The question should:
#     1. Seek insights, opinions, or advice from professional experience.
#     2. Avoid easily Googleable facts or yes/no answers.
#     3. Use natural, varied phrasing (avoid formulaic openings).
#     4. Be concise (aim for 90-110 characters on average).
#     5. Be in grammatically correct English.
    
#     Example: "What unexpected challenges have you faced in {sub_category} within {vertical}? How did you overcome them?"
#     """

In [35]:
# def generate_prompt(vertical: str, sub_category: str, tags: list) -> str:
#     """Generate a prompt based on the vertical, sub category, and tags."""
    
#     # Format tags if the list is not empty
#     if tags:
#         tags_text = ", ".join(tags)
#         tag_instructions = f", focusing on the following concepts: {tags_text} (use as many as relevant, not necessary to use all tags in one question)."
#     else:
#         tag_instructions = "."  # No tags, so just end the sentence
    
#     return f"""Generate an open-ended question about {vertical} - {sub_category}{tag_instructions}
#     The question should:
#     1. Encourage sharing insights, opinions, and advice based on professional expertise and personal experience.
#     2. Not have a single "right" answer or ask for easily Googleable facts.
#     3. Be naturally phrased and not template-based.
#     4. Be in English and grammatically correct.
#     5. Avoid yes/no answers.
#     Example: "How has [concept from {sub_category}] impacted your approach to {vertical}, and what advice would you give to others in the industry based on your experience?" """

In [36]:
def read_spreadsheet(file_path: str) -> pd.DataFrame:
    """Read the spreadsheet and return DataFrame."""
    return pd.read_excel(file_path)

# def generate_prompt(vertical: str, sub_category: str, tags: str) -> str:
#     """Generate a prompt based on the vertical, sub category, and tags."""
#     return f"""Generate an open-ended question about {vertical} - {sub_category}, focusing on the following concepts: {tags}.
#     The question should:
#     1. Encourage sharing insights, opinions, and advice based on professional expertise and personal experience.
#     2. Not have a single "right" answer or ask for easily Googleable facts.
#     3. Be naturally phrased and not template-based.
#     4. Be in English and grammatically correct.
#     5. Avoid yes/no answers.
#     Example: "How has [concept from {sub_category}] impacted your approach to {vertical}, and what advice would you give to others in the industry based on your experience?" """


In [37]:
from openai import OpenAI
def generate_question(prompt: str) -> str:
    """Use OpenAI to generate a question based on the prompt."""
    client = OpenAI(api_key=config["open_api_key"])
    response = client.chat.completions.create(
        model=config["open_ai_model_type"],
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates insightful, open-ended business questions encouraging professional insights and experiences."},
            {"role": "user", "content": prompt}
        ]
    )
    # return response.choices[0].message['content']
    return response.choices[0].message.content

In [38]:
def check_similarity_llm(question1: str, question2: str) -> bool:
    """Use LLM to evaluate similarity between two questions and return True if similarity is above 70%."""
    
    client = OpenAI(api_key=config["open_api_key"])
    
    # Prompt the model to analyze similarity and give a score
    prompt = f"""
    You are an assistant that evaluates the similarity between two questions. 
    Please analyze the following questions and provide a similarity score between 0% and 100%.
    
    Question 1: "{question1}"
    Question 2: "{question2}"
    
    Provide a similarity score (just the number) based on how similar the questions are in meaning. 
    A higher score indicates higher similarity.
    """
    
    response = client.chat.completions.create(
        model=config["open_ai_model_type"],
        messages=[
            {"role": "system", "content": "You are an expert in evaluating question similarity for redundancy."},
            {"role": "user", "content": prompt}
        ]
    )
    
    # Extract and parse the similarity score from the response
    # similarity_score = float(response.choices[0].message.content.strip('%'))


    # Sample content that contains the similarity score as a percentage
    content = response.choices[0].message.content
    
    # Extract the numeric portion and convert it to a float
    try:
        match = re.search(r'\d+(\.\d+)?', content)
        if match:
            similarity_score = float(match.group())
        else:
            similarity_score = 0.0
    except (AttributeError, ValueError) as e:
        similarity_score = 0.0
        print(f"Error extracting similarity score: {e}")
    # print(response.choices[0].message.content)
    
    # Determine if questions are similar based on the 90% threshold
    if similarity_score >= 95:
        return True  # Questions are similar
    else:
        return False  # Questions are not similar

In [39]:
def calculate_cosine_similarity(vec1, vec2):
    """Calculate the cosine similarity between two vectors."""
    # Convert lists to NumPy arrays
    a = np.array(vec1)
    b = np.array(vec2)
    
    # Calculate the cosine similarity
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    if norm_a == 0 or norm_b == 0:  # Prevent division by zero
        return 0.0
    
    return dot_product / (norm_a * norm_b)

def check_similarity(question1: str, question2: str) -> bool:
    """Check if two questions are similar using a threshold-based cosine similarity."""
    
    client = OpenAI(api_key=config["open_api_key"])
    
    # Generate embeddings for both questions
    response1 = client.embeddings.create(
        model=config["open_ai_embedding_model_type"],
        input=question1
    )
    
    response2 = client.embeddings.create(
        model=config["open_ai_embedding_model_type"],
        input=question2
    )
    
    embedding1 = response1.data[0].embedding
    embedding2 = response2.data[0].embedding
    
    # Calculate cosine similarity using the custom function
    similarity_score = calculate_cosine_similarity(embedding1, embedding2)

    print(similarity_score)
    
    # Check if similarity score is above 0.7 (70%)
    if similarity_score >= 0.9:
        return True  # Questions are similar
    else:
        return False  # Questions are not similar

In [40]:
check_similarity("How can profit and loss analyses benefit a business?", "How can profit and loss analyses benefit a business?")

1.0


True

In [41]:
# def is_question_valid(question: str, existing_questions: List[str]) -> bool:
#     """Validate the generated question."""
#     if not question:
#         return False
    
#     # Check if it's unique
#     if question.lower() in [q.lower() for q in existing_questions if isinstance(q, str)]:
#         return False
    
#     # Check if it's open-ended (not yes/no)
#     if re.match(r'^(is|are|do|does|has|have|can|could|will|would|should)\s', question.lower()):
#         return False
    
#     # Check if it ends with a question mark
#     if not question.endswith('?'):
#         return False
    
#     # Check minimum length (assuming a good question is at least 50 characters)
#     if len(question) < 50:
#         return False
    
#     return True

In [42]:
def generate_tags(question: str) -> str:
    """Use OpenAI to generate tags based on the input question."""
    client = OpenAI(api_key=config["open_api_key"])
    response = client.chat.completions.create(
        model=config["open_ai_model_type"],
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates relevant 3-4 tags based on the provided question. Please provide the tags as a comma-separated list without any symbols, such as hashtags."},
            {"role": "user", "content": question}
        ]
    )
    # Return the tags as a string without any label
    return response.choices[0].message.content.strip()  # Strip to remove any leading/trailing whitespace

In [46]:
def check_if_similar_question(new_question: str, questions_list: list) -> bool:
    """Check if the new question is similar to any questions in the existing list."""
    for entry in questions_list:
        existing_question = entry['Question']
        # Simple comparison (you may choose to improve this)
        if check_similarity_llm(new_question, existing_question) :
            return True  # Found a similar question
    return False  # No similar question found

In [47]:
def append_questions(data, n_questions):
    """Iterate through verticals and subcategories to append questions until each subcategory has 7 questions."""
    start_time = time.time()
    total_items = sum(len(sub_categories) for sub_categories in data.values())
    # Iterate over each vertical
    with tqdm(total=total_items, desc="Overall Progress", unit="subcategory", dynamic_ncols=True) as pbar:
        for vertical, sub_categories in tqdm(data.items()):
            # Iterate over each subcategory
            for sub_category, questions_list in sub_categories.items():
                # Extract tags while handling NaN values
                tags_list = [entry['Tag(s)'] for entry in questions_list if isinstance(entry['Tag(s)'], str) and entry['Tag(s)']]
                tags_list = list(set(tags_list))
                print(f"Generating for Vertical: '{vertical}', Subcategory: '{str(sub_category)}', and Tag_List: '{tags_list}'")
                
                # Count existing questions and generate new ones until the total is 7
                # while len(questions_list) < n_questions:
                count = 0
                while count < n_questions:
                    # Call generate_prompt for the subcategory
                    prompt = generate_prompt(vertical, sub_category, tags_list)
                    
                    # Generate the question
                    new_question = generate_question(prompt)
    
                    # Check for similarity before proceeding
                    # if check_if_similar_question(new_question, questions_list):
                    #     print(f"Regenerating question as it is similar to an existing question.")
                    #     continue  # Regenerate the question if it is similar
                    
                    # Generate tags for the new question
                    # generated_tags = generate_tags(new_question)
    
                    # Create a new question entry with tags, question, and publish date
                    new_question_entry = {
                        'Tag(s)': float('nan'),  # Use the generated tags
                        'Question': new_question,    # Add the generated question 
                        'Published': float('nan'),
                        'Generated_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Use current time as generated date
                    }
                    
                    # Append the new question to the subcategory's question list
                    questions_list.append(new_question_entry)
                    count+=1
    
                    print(f"Generated Question: '{new_question}'")
                    # print(f"Tags: {generated_tags}")
    
                print(f"{n_questions} questions added for the subcategory '{sub_category}' under vertical '{vertical}' .")
                # Check if the total number of questions exceeds 5
                # if len(questions_list) >= n_questions:
                #     print(f"The subcategory '{sub_category}' under vertical '{vertical}' already has {len(questions_list)} questions. No more questions will be added.")
                
                # Print a separator for better readability
                print("\n" + "*"*100 + "\n")  # Print a separator line

                # Update progress bar
                pbar.update(1)
                elapsed_time = time.time() - start_time
                avg_time = elapsed_time / pbar.n
                eta = avg_time * (total_items - pbar.n)
                pbar.set_postfix(ETA=f"{eta:.2f}s")

    return data

In [48]:
file_path = "Fiverr Answers - Question list - Oren.xlsx"
df = read_spreadsheet(file_path)
existing_questions = df['Question'].tolist()

In [49]:
df.columns

Index(['Vertical', 'Sub Category', 'Tag(s)', 'Question', 'Published'], dtype='object')

In [50]:
df.head()

,Vertical,Sub Category,Tag(s),Question,Published
0,Business Management,Business Consulting,analysis,How can profit and loss analyses benefit a bus...,2024-11-04 00:00:00
1,Business Management,Business Consulting,sustainability,What are the best practices for promoting envi...,2024-11-04 00:00:00
2,Business Management,Business Consulting,social impact,How can businesses create a positive social im...,2024-11-04 00:00:00
3,Business Management,CRM Management,"hubspot, crm, software",What are the key features of HubSpot CRM softw...,2024-11-04 00:00:00
4,Business Management,CRM Management,"salesforce, crm, software",What are the advantages of using Salesforce fo...,2024-11-04 00:00:00


In [51]:
# Initialize an empty dictionary
organized_dict = {}

# Iterate through the rows and populate the dictionary
for _, row in df.iterrows():
    vertical = row['Vertical']
    sub_category = row['Sub Category']
    
    # Create nested structure for vertical and subcategory if not exists
    if vertical not in organized_dict:
        organized_dict[vertical] = {}
    
    if sub_category not in organized_dict[vertical]:
        organized_dict[vertical][sub_category] = []
    
    # Append the question details
    organized_dict[vertical][sub_category].append({
        'Tag(s)': row['Tag(s)'],
        'Question': row['Question'],
        'Published': row['Published'],
        'Generated_date': float('nan')
    })

In [52]:
organized_dict

{'Business Management': {'Business Consulting': [{'Tag(s)': 'analysis',
    'Question': 'How can profit and loss analyses benefit a business?',
    'Published': datetime.datetime(2024, 11, 4, 0, 0),
    'Generated_date': nan},
   {'Tag(s)': 'sustainability',
    'Question': 'What are the best practices for promoting environmental sustainability in a business?',
    'Published': datetime.datetime(2024, 11, 4, 0, 0),
    'Generated_date': nan},
   {'Tag(s)': 'social impact',
    'Question': 'How can businesses create a positive social impact within their communities?',
    'Published': datetime.datetime(2024, 11, 4, 0, 0),
    'Generated_date': nan},
   {'Tag(s)': 'startups',
    'Question': 'What key performance indicators (KPIs) should a startup prioritize in its first year, and how can they be adjusted as the business scales?',
    'Published': datetime.datetime(2024, 11, 9, 0, 0),
    'Generated_date': nan},
   {'Tag(s)': nan,
    'Question': 'How do you approach competitive analysis

In [55]:
updated_organized_dict = append_questions(copy.deepcopy(organized_dict), 20)

  0%|                                                                                                                             | 0/11 [00:00<?, ?it/s]

Generating for Vertical: 'Business Management', Subcategory: 'Business Consulting', and Tag_List: '['analysis', 'social impact', 'startups', 'sustainability']'
Generated Question: 'How can startup consulting balance profit motives with social impact, without compromising sustainability?'
Generated Question: 'How do you balance gut instinct with data analysis when advising startups on sustainability and social impact?'
Generated Question: 'How do startups balance deep social impact and sustainable growth? Any surprising strategies you've seen?'
Generated Question: 'In your experience, how can startups leverage consulting to balance rapid growth with social impact strategies?'
Generated Question: 'How does embracing sustainability in startup consulting alter the traditional paths to scaling impact?'
Generated Question: 'How do you balance driving social impact and profitability in a startup while maintaining sustainability?'
Generated Question: 'How do you balance sustainable practices w

Overall Progress:   1%|▍                                                                        | 1/155 [00:28<1:13:57, 28.82s/subcategory, ETA=4438.49s]

Generated Question: 'In what unexpected ways can a consultant help a startup balance growth and sustainability, especially for social impact-oriented businesses?'
20 questions added for the subcategory 'Business Consulting' under vertical 'Business Management' .

****************************************************************************************************

Generating for Vertical: 'Business Management', Subcategory: 'CRM Management', and Tag_List: '['salesforce, crm, software', 'hubspot, crm, software']'
Generated Question: 'How has integrating Salesforce or HubSpot transformed team dynamics or customer interaction in your business?'
Generated Question: 'How do you balance AI automation in CRM systems like Salesforce or HubSpot with maintaining a personal touch?'
Generated Question: 'Have you ever seen an unusual CRM feature in HubSpot or Salesforce truly transform customer interactions? How?'
Generated Question: 'How has integrating HubSpot with other tools reshaped your approa

Overall Progress:   1%|▉                                                                        | 2/155 [00:49<1:00:33, 23.75s/subcategory, ETA=3749.90s]

...

Generated Question: 'What's the most unconventional pricing strategy you've tried in freelancing, and how did it reshape your work relationships?'
20 questions added for the subcategory 'Pricing Strategies' under vertical 'Freelancing' .

****************************************************************************************************

Generating for Vertical: 'Freelancing', Subcategory: 'Time Management', and Tag_List: '[]'
Generated Question: 'What unconventional time management trick have you discovered that truly enhances your freelancing flow?'
Generated Question: 'How do you juggle creative flexibility with strict deadlines in freelancing without burning out?'
Generated Question: 'What unusual habits or routines have you found essential in mastering time management as a freelancer?'
Generated Question: 'Have you ever discovered time savings in your freelancing routine through unconventional methods?'
Generated Question: 'How do you navigate the blurred time boundaries in freel

Overall Progress:  99%|████████████████████████████████████████████████████████████████████████▌| 154/155 [1:07:12<00:24, 24.91s/subcategory, ETA=26.18s]

Generated Question: '"How does freelancing reshape your concept of 'work hours,' and what surprising time hacks have you discovered?"'
20 questions added for the subcategory 'Time Management' under vertical 'Freelancing' .

****************************************************************************************************

Generating for Vertical: 'Freelancing', Subcategory: 'Upskilling & Diversification', and Tag_List: '[]'
Generated Question: 'In the freelancing world, how do you decide between deepening your skills or exploring new ones? Any surprising pivots or shifts in your journey?'
Generated Question: 'How has dabbling in unconventional skills or industries unexpectedly shaped your freelancing journey?'
Generated Question: 'How does unexpectedly mastering a new skill impact your freelance projects and client relationships? Any unique tales?'
Generated Question: 'How has a quirky or unconventional skill transformed your freelancing path? Any surprising client reactions?'
Genera

Overall Progress: 100%|██████████████████████████████████████████████████████████████████████████| 155/155 [1:07:33<00:00, 26.15s/subcategory, ETA=0.00s]

Generated Question: 'How have you pivoted your freelance skills unexpectedly to seize new opportunities? Any surprising success tales?'
20 questions added for the subcategory 'Upskilling & Diversification' under vertical 'Freelancing' .

****************************************************************************************************



In [313]:
# import re

# # Sample content that contains the similarity score as a percentage
# content = '35%nn'

# # Extract the numeric portion and convert it to a float
# match = re.search(r'\d+(\.\d+)?', content)
# if match:
#     similarity_score = float(match.group())
# similarity_score

In [375]:
updated_organized_dict

{'Business Management': {'Business Consulting': [{'Tag(s)': 'analysis',
    'Question': 'How can profit and loss analyses benefit a business?',
    'Published': datetime.datetime(2024, 11, 4, 0, 0),
    'Generated_date': nan},
   {'Tag(s)': 'sustainability',
    'Question': 'What are the best practices for promoting environmental sustainability in a business?',
    'Published': datetime.datetime(2024, 11, 4, 0, 0),
    'Generated_date': nan},
   {'Tag(s)': 'social impact',
    'Question': 'How can businesses create a positive social impact within their communities?',
    'Published': datetime.datetime(2024, 11, 4, 0, 0),
    'Generated_date': nan},
   {'Tag(s)': 'startups',
    'Question': 'What key performance indicators (KPIs) should a startup prioritize in its first year, and how can they be adjusted as the business scales?',
    'Published': datetime.datetime(2024, 11, 9, 0, 0),
    'Generated_date': nan},
   {'Tag(s)': nan,
    'Question': 'How do you approach competitive analysis

In [56]:
rows = []
for vertical, sub_categories in updated_organized_dict.items():
    for sub_category, question_list in sub_categories.items():
        for entry in question_list:
            rows.append({
                'Vertical': vertical,
                'Sub Category': sub_category,
                'Tag(s)': entry['Tag(s)'],
                'Question': entry['Question'],
                'Published': entry['Published'],
                'Generated_date': entry['Generated_date']
            })

# Convert rows to DataFrame and save to CSV
df = pd.DataFrame(rows)
df.to_csv("output_questions_new_20.csv", index=False)